In [1]:
import functions as volt
import pandas as pd

QQQCompanies = volt.getCompaniesETF('QQQ')['asset'].tolist()
spyCompanies = volt.getCompaniesETF('SPY')['asset'].tolist()
spyQQQSymbols = set(QQQCompanies + spyCompanies)

earningsCalendar = volt.getEarningsCalendar()
potentialUSSymbols = earningsCalendar[earningsCalendar['symbol'].str.contains(r'^[A-Za-z]+$', na=False)]

USSymbols = potentialUSSymbols[potentialUSSymbols['symbol'].isin(spyQQQSymbols)]

USSymbols.head()



,date,symbol,eps,epsEstimated,time,revenue,revenueEstimated,fiscalDateEnding,updatedFromDate
12,2024-10-11,PGR,2.65,3.33,bmo,NaN,1.878300e+10,2024-09-30,2024-10-11
83,2024-10-11,JPM,NaN,4.02,bmo,NaN,4.164930e+10,2024-09-30,2024-10-11
121,2024-10-11,BLK,NaN,10.34,bmo,NaN,5.006700e+09,2024-09-30,2024-10-10
168,2024-10-11,FAST,NaN,0.52,bmo,NaN,1.907990e+09,2024-09-30,2024-10-10
206,2024-10-11,WFC,NaN,1.27,bmo,NaN,2.040840e+10,2024-09-30,2024-10-10


In [46]:
def getSectorBetas():
    sectorList = ['XLK','XLV','XLF','XLY','XLP','XLE','XLI','XLB','XLU','XLRE','XLC']

    sectorsBeta = {}
    for sector in sectorList:
        sectorProfile = volt.getCompanyProfile(sector)['profile']
        sectorBeta = sectorProfile['beta']
        sectorName = sectorProfile['companyName'].replace(' Select Sector SPDR\xa0Fund','').replace(' Select Sector SPDR Fund','')

        sectorsBeta[sectorName] = sectorBeta

    return str(sectorsBeta)

In [47]:
getSectorBetas()

"{'Technology': 1.22, 'Health Care': 0.67, 'Financial': 1.02, 'Consumer Discretionary': 1.29, 'Consumer Staples': 0.58, 'Energy': 0.72, 'Industrial': 1.09, 'Materials': 1.17, 'Utilities': 0.74, 'The Real Estate': 1.19, 'Communication Services': 1.1}"

In [4]:
import datetime
def getMarketSnapshot():
    vix = volt.getMarketIndex('VIX')['price']
    vixWeekAgo = volt.getPrices('^VIX').iloc[5]['close']
    vixChange = (vix - vixWeekAgo) / vixWeekAgo

    Vvix = volt.getMarketIndex('VVIX')['price']
    VvixWeekAgo = volt.getPrices('^VVIX').iloc[5]['close']
    VvixChange = (Vvix - VvixWeekAgo) / VvixWeekAgo

    spHistVolatility = volt.getHistoricalVolatility('^SPX',30)
    weekAgo = datetime.datetime.today() - datetime.timedelta(7)
    spHistoricalVolatilityWeekAgo = volt.getHistoricalVolatility('^SPX',30,weekAgo)
    spChange = (spHistVolatility - spHistoricalVolatilityWeekAgo) / spHistoricalVolatilityWeekAgo
    
    returnDict = {
        'VIX': {
            'currentVIX': vix,
            'Weekly Change': str(vixChange * 100) + '%'
        },
        'S&P500': {
            'Realized Volatility': str(spHistVolatility * 100) + '%',
            'Realized Volatility Weekly Change': str(spChange * 100) + '%'
        },
        'VVIX': {
            'currentVVIX': Vvix,
            'Weekly Change': str(VvixChange * 100) + '%'
        }
    }

    return str(returnDict)

getMarketSnapshot()


"{'VIX': {'currentVIX': 21.1, 'Weekly Change': '9.838625715773038%'}, 'S&P500': {'Realized Volatility': '9.852167083706673%', 'Realized Volatility Weekly Change': '-14.506347276239767%'}, 'VVIX': {'currentVVIX': 117.43, 'Weekly Change': '1.5479072985126308%'}}"

In [8]:
def getMacroRisk():
    vix = volt.getMarketIndex('VIX')
    vixWeekAgo = volt.getPrices('^VIX').iloc[5]['close']
    vixChange = (vix['price'] - vixWeekAgo)

    returnDict = {
        'Current VIX': vix['price'],
        'Weekly Change': vixChange,
        'Weekly Change Percentage': str((vixChange / vixWeekAgo) * 100) + '%',
        '52-week High': vix['yearHigh'],
        '52-week Low': vix['yearLow']
    }
    return str(returnDict)

"{'Current VIX': 20.87, 'Weekly Change': 1.6600000000000001, 'Weekly Change Percentage': '8.64133263925039%', '52-week High': 35.05, '52-week Low': 15.53}"

In [36]:
def getHighImpactBetas():
    highImpactStocksSample = pd.concat([
        volt.getCompaniesETF('XLF')['asset'][0:15],
        volt.getCompaniesETF('XLK')['asset'][0:5],
        volt.getCompaniesETF('XLV')['asset'][0:3]
    ])

    returnDict = {}
    highImpactStocks = highImpactStocksSample.sample(5)
    for stock in highImpactStocks:
        profile = volt.getCompanyProfile(stock)['profile']
        returnDict[f"{profile['companyName']} ({profile['symbol']})"] = profile['beta']

    return str(returnDict)

In [67]:
def getInterestRateEnviroment():
    tresuryRates = volt.getTresuryRates()
    today = tresuryRates.iloc[0].to_dict()
    week = tresuryRates.iloc[5].to_dict()

    month1 = tresuryRates.iloc[abs(pd.to_datetime(tresuryRates['date']) - pd.Timestamp(datetime.datetime.today() - datetime.timedelta(30))).idxmin()].to_dict()

    month3 = tresuryRates.iloc[-1].to_dict()

    return {
        'Treasury Rates': {
            'Today': today,
            'One Week Ago': week,
            'One Month Ago': month1,
            'Three Months Ago': month3
        }
    }

In [45]:
import datetime

def getHighImpactEconomicEvents():
    economicCalendar = volt.getEconomicsCalendar()

    highImpactEvents = economicCalendar[economicCalendar['impact'] == 'High'].reset_index(drop=True)
    highImpactEvents['date'] = pd.to_datetime(highImpactEvents['date'])

    future_date = pd.Timestamp(datetime.datetime.today() + datetime.timedelta(days=7))
    index = abs(highImpactEvents['date'] - future_date).idxmin() + 1 

    highImpactEventsThisWeek = highImpactEvents[0:index]
    highImpactEventsThisWeek.drop(columns=['change','actual','impact','changePercentage','unit'],inplace=True)

    returnDict = {}
    for index, row in highImpactEventsThisWeek.iterrows():
        rowDict = row.to_dict()
        rowDict['date'] = str(rowDict['date'])
        event = rowDict.pop('event')
        returnDict[event] = rowDict

    return str(returnDict)
